# <span style="color:#0c0c0c"><strong>HR ANALYTICS</strong></span>

### <span style="color:#808080"><strong>Case Study</strong></span>

Bajaj Motors, a fictional French-based alternative energy vehicle manufacturer, aims to analyze their HR data and come up with insights and recommendations to improve employee retention.

For this case study, the PACE (Plan - Analyze - Construct - Execute) framework will be utilized.

# <span style="color:#4784f0"><strong>PLAN STAGE</strong></span>

## <span style="color:#808080">Understanding the business scenario and problem</span>

The HR department at Bajaj Motors wants to take some initiatives to improve employee satisfaction levels at the company. They collected data from employees and is need of a data analytics professional that will provide data-driven suggestions based on the data. They have the following question: what’s likely to make the employee leave the company?

This project will focus on analyzing the data collected by the HR department and on building a model that will predict whether a employee will leave or stay in the company.

The company will also benefit by identifying factors that affect their decision to leave. It is time-consuming and expensive to find, interview, and hire new employees; thus, increasing employee retention will be advantageous to the company.


## <span style="color:#808080">Familiarizing with the HR dataset</span>

The dataset used in this case study is located on [Kaggle](https://www.kaggle.com/datasets/mfaisalqureshi/hr-analytics-and-job-prediction?select=HR_comma_sep.csv), and it contains 15,000 rows and 10 columns for the variables listed below. 

Variable  |Description |
-----|-----|
satisfaction_level|Employee-reported job satisfaction level [0&ndash;1]|
last_evaluation|Score of employee's last performance review [0&ndash;1]|
number_project|Number of projects employee contributes to|
average_monthly_hours|Average number of hours employee worked per month|
time_spend_company|How long the employee has been with the company (years)
Work_accident|Whether or not the employee experienced an accident while at work
left|Whether or not the employee left the company
promotion_last_5years|Whether or not the employee was promoted in the last 5 years
Department|The employee's department
salary|The employee's salary (U.S. dollars)

## <span style="color:#808080">Import Packages</span>

In [ ]:
# Import packages
### YOUR CODE HERE ### 

# For data manipulation
import numpy as np
import pandas as pd

# For data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# For displaying all of the columns in dataframes
pd.set_option('display.max_columns', None)

# For data modeling
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from xgboost import plot_importance

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# For metrics and helpful functions
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score,\
f1_score, confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.tree import plot_tree

# For saving models
import pickle

## <span style="color:#808080">Load Dataset</span>

In [ ]:
df0 = pd.read_csv("/kaggle/input/hr-analytics-and-job-prediction/HR_comma_sep.csv")

df0.head()

## <span style="color:#808080">Initial Exploratory Data Analysis (EDA)</span>

Gather basic information about the data.

In [ ]:
df0.info()

Gather descriptive statistics about the data.

In [ ]:
df0.describe(include='all')

As a data cleaning step, the columns were renamed and standardized so that they are all concise, correctly spelled and in `snake_case`.

In [ ]:
df0 = df0.rename(columns={'Work_accident': 'work_accident',
                          'average_montly_hours': 'average_monthly_hours',
                          'time_spend_company': 'tenure',
                          'Department': 'department'})
df0.columns

Check for missing values in the dataset.

In [ ]:
df0.isna().sum()

The dataset has no missing values.


Next, check for duplicate entries in the data.

In [ ]:
df0.duplicated().sum()

There are 3,008 rows that contain duplicates, which is about 20% of the data. Inspect some of the rows containing duplicates.

In [ ]:
df0[df0.duplicated()].head()

With 10 columns and several continuous variables among them, it is likely that these observations are invalid so they will be dropped.

In [ ]:
df1 = df0.drop_duplicates(keep='first')
df1.head()

Check for outliers in the data.

In [ ]:
plt.figure(figsize=(6,3))
plt.title('Boxplot to detect outliers for tenure', fontsize=10)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
sns.boxplot(x=df1['tenure'])
plt.show()

The diagram shows that there are outliers in the `tenure` variable. Next, we will investigate the outliers in this particular column.

In [ ]:
percentile25 = df1['tenure'].quantile(0.25)
percentile75 = df1['tenure'].quantile(0.75)
iqr = percentile75 - percentile25

upper_limit = percentile75 + 1.5 * iqr
lower_limit = percentile25 - 1.5 * iqr
print("Lower limit:", lower_limit)
print("Upper limit:", upper_limit)

outliers = df1[(df1['tenure'] > upper_limit) | (df1['tenure'] < lower_limit)]

print("Number of rows in the data with outliers in `tenure`:", len(outliers))

Some model types are susceptible to outliers. When we get to the stage of constructing the model, we will consider whether to remove or keep these outliers based on the type of model we decide to use.

# <span style="color:#ec4434"><strong>ANALYZE STAGE</strong></span>

## <span style="color:#808080">Exploratory Data Analysis (EDA)</span>

Calculate how many employees left and what percentage of all employees this figure represents.

In [ ]:
print(df1['left'].value_counts())
print()

print('Percentages:')
print((df1['left'].value_counts(normalize=True))*100)

There is an imbalance in the classes but the majority class does not go beyond 90%. Since the imbalance is not extreme, we will continue without modifying the class balance.

## <span style="color:#808080">Data Visualizations</span>
We will create visualizations to examine the variables and their relationships to other variables variables in the dataset.

Let us start looking at the `average_monthly_hours` and `number_project`.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (22,8))

# Create boxplot showing `average_monthly_hours` distributions for `number_project`, comparing employees who stayed versus those who left
sns.boxplot(data=df1, x='average_monthly_hours', y='number_project', hue='left', orient="h", ax=ax[0])
ax[0].invert_yaxis()
ax[0].set_title('Monthly hours by number of projects', fontsize='14')

# Create histogram showing distribution of `number_project`, comparing employees who stayed versus those who left
tenure_stay = df1[df1['left']==0]['number_project']
tenure_left = df1[df1['left']==1]['number_project']
sns.histplot(data=df1, x='number_project', hue='left', multiple='dodge', shrink=2, ax=ax[1])
ax[1].set_title('Number of projects histogram', fontsize='14')

plt.show()

The mean hours of each group (those who stayed and those who left) increases with the number of projects worked. Naturally, people who are working on more projects would also be working for longer hours. However, a other things are notable from the visualizations.

1. We see two prominent groups of employees who exited the company: (A) those whose average monthly hours are less than their peers, and (B) those who worked much more hours than their peers. Group A possibly includes those who were fired or employees who had already given their notice and were assigned fewer tasks because they were already on their way out of the company. For those in group B, it is reasonable to infer that these are the ones that quit. The employees in group B might have been the largest contributors to their projects. 

2. All employees who were assigned with seven projects left the company, and the interquartile ranges of this group and those who left with six projects was about 255 to 295 hours/week, which is much more than any other group. 

3. The optimal number of projects for employees to work on seems to be 3&ndash;4 since the ratio of left/stayed is very small for these groups.

4. Assuming a work week of 40 hours and two weeks of vacation per year, the average number of working hours per month of employees working Monday&ndash;Friday `= 50 weeks * 40 hours per week / 12 months = 166.7 hours per month`. This means that, aside from the employees who worked on only two projects, every group&mdash;even those who did not leave the company&mdash;worked considerably more hours than this. It seems that employees in the company are overworked.

5. In addition, the increase in mean hours for people working on more projects were much more evident for the people who left than those people who stayed. This indicates that the people who quitted were even much more overworked that those who did not.

Validate and confirm that all employees with seven projects left.

In [ ]:
df1[df1['number_project']==7]['left'].value_counts()

This confirms that all employees with 7 projects did leave. 

Next, examine the average monthly hours versus the satisfaction levels. 

In [ ]:
# Create scatterplot of `average_monthly_hours` versus `satisfaction_level`, 
# comparing employees who stayed versus those who left
plt.figure(figsize=(16, 9))
sns.scatterplot(data=df1, x='average_monthly_hours', y='satisfaction_level', hue='left', alpha=0.4)
plt.axvline(x=166.7, color='#ff6361', label='166.7 hrs./mo.', ls='--')
plt.legend(labels=['166.67 hrs./mo.', 'stayed', 'left'])
plt.title('Monthly hours by last evaluation score', fontsize='12')

The diagram shows that there is a significant number of employees who worked about 240&ndash;315 hours per month, or over 75 hours per week for a whole year. Most likely, this is related to their satisfaction levels being close to zero. Secondly, the plot shows another group of people who left, those who had more normal working hours. Even so, their satisfaction was only around 0.4. It is possible they felt pressured to work more, given that many of their peers are working more hours. There is another group who worked about 210&ndash;280 hours per month, and they had satisfaction levels ranging about 0.7&ndash;0.9. 

The strange shape of the distributions in the dataset is indicative of data manipulation or synthetic data.

For the next visualizations, it might be interesting to visualize satisfaction levels and tenure.

In [ ]:
plt.figure(figsize=(4,3))
# Create histogram showing distribution of `satisfaction_level`
tenure_left = df1[df1['left']==1]['satisfaction_level']
sns.histplot(data=tenure_left, color ='#e1812b')
plt.title('Satisfaction histogram for those who left', fontsize='10')

plt.show()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (22,8))

# Create boxplot showing distributions of `satisfaction_level` by tenure, comparing employees who stayed versus those who left
sns.boxplot(data=df1, x='satisfaction_level', y='tenure', hue='left', orient="h", ax=ax[0])
ax[0].invert_yaxis()
ax[0].set_title('Satisfaction by Tenure', fontsize='14')

# Create histogram showing distribution of `tenure`, comparing employees who stayed versus those who left
tenure_stay = df1[df1['left']==0]['tenure']
tenure_left = df1[df1['left']==1]['tenure']
sns.histplot(data=df1, x='tenure', hue='left', multiple='dodge', shrink=5, ax=ax[1])
ax[1].set_title('Tenure Histogram', fontsize='14')

plt.show()

Notable observations based from the visualizations:

- There are two general categories for employees who left: (A) dissatisfied employees with short tenures and, (B) very satisfied employees with medium-length tenures.
- Employees who left with tenure of 4 years have an unusually low satisfaction level. It is worth investigating company policies and changes in these policies that might have affected employees. 
- Employees with more than 7 years tenure did not leave and their satisfaction levels were also high. 
- There are relatively few longer-tenured employees. It is possible that they are the higher-ranking and higher-paid employees.

Next, calculate the mean and median satisfaction scores of employees who left and those who did not.

In [ ]:
df1.groupby(['left'])['satisfaction_level'].agg(['mean','median'])

The mean and median satisfaction scores of employees who left are lower than those of employees who stayed, which is as expected. Among the employees who stayed, the mean satisfaction score is slightly below the median score. This indicates a left-skewed satisfaction level distribution

Next, examine salary levels for different tenures.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (22,8))

# Define short-tenured employees
tenure_short = df1[df1['tenure']<6]

# Define long-tenured employees
tenure_long = df1[df1['tenure']>6]

# Plot short-tenured histogram
sns.histplot(data=tenure_short, x='tenure', hue='salary', discrete=1, 
             hue_order=['low', 'medium', 'high'], multiple='dodge', shrink=.5, ax=ax[0])
ax[0].set_title('Salary Histogram by Tenure: Short-tenured', fontsize='14')

# Plot long-tenured histogram
sns.histplot(data=tenure_long, x='tenure', hue='salary', discrete=1, 
             hue_order=['low', 'medium', 'high'], multiple='dodge', shrink=.4, ax=ax[1])
ax[1].set_title('Salary Histogram by Tenure: Long-tenured', fontsize='14');

Long-tenured employees were not majorly comprised of higher-paid employees. 

Next, explore whether there is a correlation between working long hours and receiving high evaluation scores. Create a scatterplot of `average_monthly_hours` versus `last_evaluation`.

In [ ]:
# Create scatterplot of `average_monthly_hours` versus `last_evaluation`
plt.figure(figsize=(16, 9))
sns.scatterplot(data=df1, x='average_monthly_hours', y='last_evaluation', hue='left', alpha=0.4)
plt.axvline(x=166.7, color='#ff6361', label='166.7 hrs./mo.', ls='--')
plt.legend(labels=['166.7 hrs./mo.', 'left', 'stayed'])
plt.title('Monthly Hours by Last Evaluation Score', fontsize='12')

Notable observations:
- There are two groups of employees who left: well-performing employees who are overworked and low-performing employees who worked slightly below 167 hours. 
- Working long hours does not guarantee a good evaluation score.
- Most of the employees in this company work well over 167 hours per month.

Next, examine whether employees who worked very long hours were promoted in the last five years.

In [ ]:
# Create plot to examine relationship between `average_monthly_hours` and `promotion_last_5years`
plt.figure(figsize=(16, 3))
sns.scatterplot(data=df1, x='average_monthly_hours', y='promotion_last_5years', hue='left', alpha=0.4)
plt.axvline(x=166.7, color='#ff6361', ls='--')
plt.legend(labels=['166.7 hrs./mo.', 'left', 'stayed'])
plt.title('Monthly Hours by Promotion in the Last 5 years', fontsize='12')

Observations from the plot:
- the employees who left were working the longest hours
- only few employees who were promoted in the last five years left
- only few employees who worked the most hours were promoted

Next step is to inspect how the employees who left are distributed across departments.

In [ ]:
df1["department"].value_counts()

In [ ]:
# Create stacked histogram to compare department distribution of employees who left to that of employees who didn't
plt.figure(figsize=(11,8))
sns.histplot(data=df1, x='department', hue='left', discrete=1, 
             hue_order=[0, 1], multiple='dodge', shrink=.5)
plt.xticks(rotation=45)
plt.title('Counts of Stayed/Left by Department', fontsize=12)
plt.ylabel('Employee Count')
plt.xlabel('Department')
plt.show()

The proportion of employees who left to those who stayed are similar to all departments. 

Lastly, check for correlations between variables in the data.

In [ ]:
# Plot a correlation heatmap
plt.figure(figsize=(16, 9))
heatmap = sns.heatmap(df0[['satisfaction_level', 'last_evaluation', 'number_project', 'average_monthly_hours', 'tenure', 'work_accident', 'left','promotion_last_5years']].corr(), vmin=-1, vmax=1, annot=True, cmap=sns.color_palette("YlGnBu", as_cmap=True))
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':14}, pad=12)

Based from the correlation heatmap, the number of projects, monthly hours, and evaluation scores have some positive correlation with each other. Satisfaction level is negatively correlated with the tendency of an employee leaving.

## <span style="color:#808080">Insights</span>

It can be inferred that **poor management** causes the employees to leave the company. Leaving is correlated to **working longer hours**, **more assigned projects**, and **lower satisfaction levels**. This is expected because it can be disappointing to work long hours and not receive any promotion or good evaluation scores. There is a significant number of employees at this company who are most likely burnt out. It also appears that beyond six years at the company, employees tend to stay. 

# <span style="color:#f3a909"><strong>CONSTRUCT STAGE</strong></span>

## <span style="color:#808080">Identify Type of Prediction Task</span>

The goal is to predict whether an employee leaves the company, which is a categorical outcome variable. So this task involves classification. More specifically, this involves binary classification, since the outcome variable `left` can be either 1 (indicating employee left) or 0 (indicating employee did not leave). 

Since the variable to be predicted is categorical, a Logistic Regression model or a Tree-based Machine Learning model is appropriate for this project.

## <span style="color:#808080">Logistic Regression Model</span>

First, encode the two non-numeric variables: `department` and `salary`. The variable `salary` is ordinal, meaning, there is a heirarchy to the categories; therefore, it is better to convert the levels to numeric values, 0&ndash;2.

In [ ]:
df1['salary'].head()

In [ ]:
df_enc = df1.copy()

# Dummy encode the `department` column
df_enc = pd.get_dummies(df_enc, drop_first=False, columns=['department'])

# Encode the `salary` column as an ordinal numeric category
df_enc['salary'] = (df_enc['salary'].astype('category').cat.set_categories(['low', 'medium', 'high']).cat.codes)

df_enc.head()

In [ ]:
# Create a heatmap to visualize how correlated variables are
plt.figure(figsize=(8, 6))
sns.heatmap(df_enc[['satisfaction_level', 'last_evaluation', 'number_project', 'average_monthly_hours', 'tenure']]
            .corr(), annot=True, cmap=sns.color_palette("YlGnBu", as_cmap=True))
plt.title('Heatmap of the Dataset')
plt.show()

In [ ]:
# Create a stacked bart plot to visualize number of employees across department, comparing those who left with those who didn't
# In the legend, 0 (purple color) represents employees who did not leave, 1 (red color) represents employees who left
pd.crosstab(df1['department'], df1['left']).plot(kind ='bar',color='cr')
plt.title('Counts of Employees Who Left Versus Stayed Across Each Department', fontsize=12)
plt.ylabel('Employee Count')
plt.xlabel('Department')
plt.show()

Logistic regression models are susceptible to outliers; therefore, it is recommended to remove the outliers in the `tenure` column, as identified in the Plan Stage.

In [ ]:
# Select rows without outliers in `tenure` and save resulting dataframe in a new variable
df_logreg = df_enc[(df_enc['tenure'] >= lower_limit) & (df_enc['tenure'] <= upper_limit)]

df_logreg.head()

Next, isolate the outcome variable as `y` and select the features `X` to be used in the model.

In [ ]:
# Isolate the outcome variable
y = df_logreg['left']

# Select the features you want to use in your model
X = df_logreg.drop('left', axis=1)

Split the data into training set and testing set and since the classes are imbalanced, data is stratified based on the values of the outcome variable. Assign a value in the `random_state` for reproducibility.

In [ ]:
# Split the data into training set and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=42)

Next, construct the logistic regression model and plot a confusion matrix to visualize the results of the model after it was used to make predictions using the test set.

In [ ]:
# Construct a logistic regression model and fit it to the training dataset
log_clf = LogisticRegression(random_state=42, max_iter=500).fit(X_train, y_train)

# Use the logistic regression model to get predictions on the test set
y_pred = log_clf.predict(X_test)

# Compute values for confusion matrix
log_cm = confusion_matrix(y_test, y_pred, labels=log_clf.classes_)

# Create display of confusion matrix
log_disp = ConfusionMatrixDisplay(confusion_matrix=log_cm, 
                                  display_labels=log_clf.classes_)

# Plot confusion matrix
log_disp.plot(values_format='', cmap=plt.cm.YlGnBu)

plt.show()

True negatives: Employees who did not leave that the model correctly predicted; upper-left quadrant

False positives: Employees who did not leave the model incorrectly predicted as leaving; upper-right quadrant

False negatives: Employees who actually left that the model inaccurately predicted as staying; bottom-left quadrant

True positives: Employees who left the company that the model accurately predicted; bottom-right quadrant

A perfect model would yield all true negatives and true positives, and no false negatives or false positives.

To summarize the results and evaluate the performance of the logistic regression model in terms of the key metrics, create a classification report that includes precision, recall, f1-score, and accuracy.

In [ ]:
# Create classification report for logistic regression model
target_names = ['Predicted would not leave', 'Predicted would leave']
print(classification_report(y_test, y_pred, target_names=target_names))

The classification report above shows that the Logistic Regression model was able to achieve a precision of 86%, recall of 93%, f1-score of 90% and accuracy of 82%.

The model performance in predicting employee retention is good; however, the logistic regression model assumes linearity, independent observations, and no multicollinearity. Also, recall that we removed outliers and the outliers might have been valid data points. Therefore, it is recommended to consider tree-based models, which do not have these assumptions and are not affected by extreme outliers.

## <span style="color:#808080">Tree-based Model</span>

This modeling approach will cover implementation of Decision Tree, Random Forest, and XGBoost.

Firstly, isolate the outcome variable as `y` and select the features `X` to be used in the model.

In [ ]:
# Isolate the outcome variable
y = df_logreg['left']

# Select the features you want to use in your model
X = df_logreg.drop('left', axis=1)

Split the data into training set and testing set and since the classes are imbalanced, data is stratified based on the values of the outcome variable. Assign a value in the `random_state` for reproducibility.

In [ ]:
# Split the data into training set and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=42)

For this project, we will use F1 as the metric to refit the models since the class is imbalanced and because of its easier interpretability, which in some cases can be a deciding factor for stakeholder decisions.

We will also pickle our fit models for saving and quicker access. Define a path for saving the model, and define functions to pickle the model and read in the model.

In [ ]:
# Define a path to the folder where you want to save the model
path = '/kaggle/working/'

In [ ]:
def write_pickle(path, model_object, save_as:str):
    '''
    In: 
        path:         path of folder where you want to save the pickle
        model_object: a model you want to pickle
        save_as:      filename for how you want to save the model

    Out: A call to pickle the model in the folder indicated
    '''    

    with open(path + save_as + '.pickle', 'wb') as to_write:
        pickle.dump(model_object, to_write)

In [ ]:
def read_pickle(path, saved_model_name:str):
    '''
    In: 
        path:             path to folder where you want to read from
        saved_model_name: filename of pickled model you want to read in

    Out: 
        model: the pickled model 
    '''
    with open(path + saved_model_name + '.pickle', 'rb') as to_read:
        model = pickle.load(to_read)

    return model

### <span style="color:#808080"><strong>Round 1</strong></span>

### <span style="color:#808080">Decision Tree</span>

Construct a Decision Tree and with Cross Validation, through GridSearch, to exhaustively search for the best model parameters.

In [ ]:
# Instantiate model
tree = DecisionTreeClassifier(random_state=0)

# Assign a dictionary of hyperparameters to search over
cv_params = {'max_depth':[4, 6, 8, None],
             'min_samples_leaf': [2, 5, 1],
             'min_samples_split': [2, 4, 6]
             }

# Assign a dictionary of scoring metrics to capture
scoring = {'accuracy', 'precision', 'recall', 'f1', 'roc_auc'}

# Instantiate GridSearch
tree1 = GridSearchCV(tree, cv_params, scoring=scoring, cv=4, refit='f1')

In [ ]:
%%time
# Fit the decision tree model to the training data.
tree1.fit(X_train, y_train) #Wall time: ~5s

In [ ]:
# Write pickle
write_pickle(path, tree1, 'hr_tree1')

In [ ]:
# Read pickle
tree1 = read_pickle(path, 'hr_tree1')

In [ ]:
# Check best parameters
tree1.best_params_

In [ ]:
# Check best F1 score on CV
tree1.best_score_

The F1 score is strong, indicating that the model can predict employees who will leave accurately.

In [ ]:
def make_results(model_name:str, model_object, metric:str):
    '''
    Arguments:
        model_name (string): what you want the model to be called in the output table
        model_object: a fit GridSearchCV object
        metric (string): precision, recall, f1, accuracy, or auc
  
    Returns a pandas df with the F1, recall, precision, accuracy, and auc scores
    for the model with the best mean 'metric' score across all validation folds.  
    '''

    # Create dictionary that maps input metric to actual metric name in GridSearchCV
    metric_dict = {'auc': 'mean_test_roc_auc',
                   'precision': 'mean_test_precision',
                   'recall': 'mean_test_recall',
                   'f1': 'mean_test_f1',
                   'accuracy': 'mean_test_accuracy'
                  }

    # Get all the results from the CV and put them in a df
    cv_results = pd.DataFrame(model_object.cv_results_)

    # Isolate the row of the df with the max(metric) score
    best_estimator_results = cv_results.iloc[cv_results[metric_dict[metric]].idxmax(), :]

    # Extract Accuracy, precision, recall, and f1 score from that row
    auc = best_estimator_results.mean_test_roc_auc
    f1 = best_estimator_results.mean_test_f1
    recall = best_estimator_results.mean_test_recall
    precision = best_estimator_results.mean_test_precision
    accuracy = best_estimator_results.mean_test_accuracy
  
    # Create table of results
    table = pd.DataFrame()
    table = pd.DataFrame({'model': [model_name],
                          'precision': [precision],
                          'recall': [recall],
                          'F1': [f1],
                          'accuracy': [accuracy],
                          'auc': [auc]
                        })
  
    return table

In [ ]:
# Get all CV scores
tree_cv_results = []
tree_cv_results = make_results('Decision Tree CV 1', tree1, 'f1')
tree_cv_results

Strong scores indicates good model performance. Decision trees can be vulnerable to overfitting; therefore, it is wise to construct a random forest for comparison.

### <span style="color:#808080">Random Forest</span>

Construct a Random Forest and with Cross Validation, through GridSearch, to exhaustively search for the best model parameters.

In [ ]:
# Instantiate model
rf = RandomForestClassifier(random_state=0)

# Assign a dictionary of hyperparameters to search over
cv_params = {'max_depth': [3,5, None], 
             'max_features': [1.0],
             'max_samples': [0.7, 1.0],
             'min_samples_leaf': [1,2,3],
             'min_samples_split': [2,3,4],
             'n_estimators': [30, 50],
             }  

# Assign a dictionary of scoring metrics to capture
scoring = {'accuracy', 'precision', 'recall', 'f1', 'roc_auc'}

# Instantiate GridSearch
rf1 = GridSearchCV(rf, cv_params, scoring=scoring, cv=4, refit='f1')

In [ ]:
%%time
# Fit the random forest model to the training data.
rf1.fit(X_train, y_train) #Wall time: ~29min

In [ ]:
# Write pickle
write_pickle(path, rf1, 'hr_rf1')

In [ ]:
# Read pickle
rf1 = read_pickle(path, 'hr_rf1')

In [ ]:
# Check best params
rf1.best_params_

In [ ]:
# Check best F1 score on CV
rf1.best_score_

In [ ]:
# Get all CV scores using the make_results() function defined earlier.
rf1_cv_results = make_results('Random Forest CV 1', rf1, 'f1')
tree_cv_results = pd.concat([tree_cv_results, rf1_cv_results], axis=0)
tree_cv_results = tree_cv_results.reset_index(drop=True)
tree_cv_results

In [ ]:
# In case a row is to be dropped, input index number(s) in drop()
#tree_cv_results = tree_cv_results.drop([5,6], axis=0).reset_index(drop=True)
#tree_cv_results

Based on the evaluation scores, the random forest is generally a better model than the decision tree. 

Next, check if XGBoost is a better model than the two approaches implemented thus far.

### <span style="color:#808080">XGBoost</span>

Construct an XGBoost model and with Cross Validation, through GridSearch, to exhaustively search for the best model parameters.

In [ ]:
# 1. Instantiate the XGBoost classifier
xgb = XGBClassifier(objective='binary:logistic', random_state=0)

# 2. Create a dictionary of hyperparameters to tune
cv_params = {'learning_rate': [0.1,0.2,0.3],
             'max_depth': [1,3,5,7],
             'min_child_weight': [0.2,0.5,0.9,1,2,5],
             'n_estimators': [30,50]
             }

# 3. Define a dictionary of scoring metrics to capture
scoring = {'accuracy', 'precision', 'recall', 'f1', 'roc_auc'}

# 4. Instantiate the GridSearchCV object
xgb1 = GridSearchCV(xgb, cv_params, scoring=scoring, cv=4, refit='f1')

In [ ]:
%%time
# Fit the XGBoost model to the training data.
xgb1.fit(X_train, y_train) #Wall time: ~5min

In [ ]:
# Write pickle
write_pickle(path, xgb1, 'hr_xgb1')

In [ ]:
# Read pickle
xgb1 = read_pickle(path, 'hr_xgb1')

In [ ]:
# Examine best parameters
xgb1.best_params_

In [ ]:
# Examine best score
xgb1.best_score_

In [ ]:
# Call 'make_results()' on the GridSearch object
xgb1_cv_results = make_results('XGB CV 1', xgb1, 'f1')
tree_cv_results = pd.concat([tree_cv_results, xgb1_cv_results], axis=0)
tree_cv_results = tree_cv_results.reset_index(drop=True)
tree_cv_results

In [ ]:
# In case a row is to be dropped, input index number(s) in drop()
#tree_cv_results = tree_cv_results.drop([5,6], axis=0).reset_index(drop=True)
#tree_cv_results

It appears that the Random Forest model outperforms Decision Tree and XGBoost.

Define a function that will get all the scores of a model's performance on the test set.

In [ ]:
def get_scores(model_name:str, model, X_test_data, y_test_data):
    '''
    Generate a table of test scores.

    In: 
        model_name (string):  How you want your model to be named in the output table
        model:                A fit GridSearchCV object
        X_test_data:          numpy array of X_test data
        y_test_data:          numpy array of y_test data

    Out: pandas df of precision, recall, f1, accuracy, and AUC scores for your model
    '''

    preds = model.best_estimator_.predict(X_test_data)

    auc = roc_auc_score(y_test_data, preds)
    accuracy = accuracy_score(y_test_data, preds)
    precision = precision_score(y_test_data, preds)
    recall = recall_score(y_test_data, preds)
    f1 = f1_score(y_test_data, preds)

    table = pd.DataFrame({'model': [model_name],
                          'precision': [precision], 
                          'recall': [recall],
                          'f1': [f1],
                          'accuracy': [accuracy],
                          'AUC': [auc]
                         })
  
    return table

Use the best performing model to predict on the test set.

In [ ]:
# Get predictions on test data
rf1_test_scores = get_scores('Random Forest CV 1', rf1, X_test, y_test)
rf1_test_scores

It appears that the Random Forest model performed even better on the test data. This model seems to be a strong model and since the test set was only used for the Random Forest model, we can be confident that this model will perform accurately on new and unseen data.

### <span style="color:#808080"><strong>Round 2</strong></span>

The very high evaluation scores might be indicative of overfitting or data leakage, or when the data used to train is not the expected data to be received once the model is deployed. It is very likely that the company will not have satisfaction scores for all employees. It is also possible for the `average_monthly_hours` column to be a source of data leakage. If the employees has already decided to quit or have already been identified as people to be fired, they make be working fewer hours than their peers.

For the second round of Contruct Stage, feature engineering will be incorporated to improve the models.

### <span style="color:#808080">Feature Engineering</span>

Drop the `satisfaction_level` column and create a new feature called `overworked`. This feature is a binary variable and will identify the overworked employees based on their `average_monthly_hours`.

In [ ]:
# Drop `satisfaction_level` and save resulting dataframe in new variable
df2 = df_enc.drop('satisfaction_level', axis=1)

df2.head()

In [ ]:
# Create `overworked` column. For now, it's identical to average monthly hours.
df2['overworked'] = df2['average_monthly_hours']

# Inspect max, min and mean average_monthly_hours values
print('Max hours:', df2['overworked'].max())
print('Min hours:', df2['overworked'].min())
print('Mean hours:', df2['overworked'].mean())

As identified earlier, 166.7 is approximately number of monthly hours for an employee who works 50 weeks annually, 5 days per week, and 8 hours per day. For this project, we will define overworked as someone who works more than 200 hours per month on average.

In [ ]:
# Define `overworked` as working > 200 hrs/week
df2['overworked'] = (df2['overworked'] > 200).astype(int)

df2['overworked'].head()

In [ ]:
# Drop the `average_monthly_hours` column
df2 = df2.drop('average_monthly_hours', axis=1)

df2.head()

Isolate the target variable and the features. Then, split the data into training and testing sets.

In [ ]:
# Isolate the outcome variable
y = df2['left']

# Select the features
X = df2.drop('left', axis=1)

# Create test data and stratify based on y
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=0)

### <span style="color:#808080">Decision Tree</span>

Construct a Decision Tree and with Cross Validation, through GridSearch, to exhaustively search for the best model parameters.

In [ ]:
# Instantiate model
tree = DecisionTreeClassifier(random_state=0)

# Assign a dictionary of hyperparameters to search over
cv_params = {'max_depth':[4, 6, 8, None],
             'min_samples_leaf': [2, 5, 1],
             'min_samples_split': [2, 4, 6]
             }

# Assign a dictionary of scoring metrics to capture
scoring = {'accuracy', 'precision', 'recall', 'f1', 'roc_auc'}

# Instantiate GridSearch
tree2 = GridSearchCV(tree, cv_params, scoring=scoring, cv=4, refit='f1')

In [ ]:
%%time
# Fit the Decision Tree model to the training data.
tree2.fit(X_train, y_train) #Wall time: ~4s

In [ ]:
# Write pickle
write_pickle(path, tree2, 'hr_tree2')

In [ ]:
# Read pickle
tree2 = read_pickle(path, 'hr_tree2')

In [ ]:
# Check best params
tree2.best_params_

In [ ]:
# Check best F1 score on CV
tree2.best_score_

The model still performs very well despite using few features.

In [ ]:
# Get all CV scores
tree2_cv_results = make_results('Decision Tree CV 2', tree2, 'f1')
tree_cv_results = pd.concat([tree_cv_results, tree2_cv_results], axis=0)
tree_cv_results = tree_cv_results.reset_index(drop=True)
tree_cv_results

In [ ]:
# In case a row is to be dropped, input index number(s) in drop()
#tree_cv_results = tree_cv_results.drop([5,6], axis=0).reset_index(drop=True)
#tree_cv_results

As expected, all of the scores fell since fewer features were selected for this round; however, the scores are still very good.

### <span style="color:#808080">Random Forest</span>

Construct a Random Forest and with Cross Validation, through GridSearch, to exhaustively search for the best model parameters.

In [ ]:
# Instantiate model
rf = RandomForestClassifier(random_state=0)

# Assign a dictionary of hyperparameters to search over
cv_params = {'max_depth': [3,5, None], 
             'max_features': [1.0],
             'max_samples': [0.7, 1.0],
             'min_samples_leaf': [1,2,3],
             'min_samples_split': [2,3,4],
             'n_estimators': [30, 50],
             }  

# Assign a dictionary of scoring metrics to capture
scoring = {'accuracy', 'precision', 'recall', 'f1', 'roc_auc'}

# Instantiate GridSearch
rf2 = GridSearchCV(rf, cv_params, scoring=scoring, cv=4, refit='f1')

In [ ]:
%%time
# Fit the Random Forest model to the training data.
rf2.fit(X_train, y_train) #Wall time ~23min

In [ ]:
# Write pickle
write_pickle(path, rf2, 'hr_rf2')

In [ ]:
# Read in pickle
rf2 = read_pickle(path, 'hr_rf2')

In [ ]:
# Check best params
rf2.best_params_

In [ ]:
# Check best F1 score on CV
rf2.best_score_

In [ ]:
rf2_cv_results = make_results('Random Forest CV 2', rf2, 'f1')
tree_cv_results = pd.concat([tree_cv_results, rf2_cv_results], axis=0)
tree_cv_results = tree_cv_results.reset_index(drop=True)
tree_cv_results

In [ ]:
# In case a row is to be dropped, input index number(s) in drop()
#tree_cv_results = tree_cv_results.drop([5,6], axis=0).reset_index(drop=True)
#tree_cv_results

Again, the scores for the second round were a little lower than the first round.

### <span style="color:#808080">XGBoost</span>

Construct an XGBoost model and with Cross Validation, through GridSearch, to exhaustively search for the best model parameters.

In [ ]:
# 1. Instantiate the XGBoost classifier
xgb = XGBClassifier(objective='binary:logistic', random_state=0)

# 2. Create a dictionary of hyperparameters to tune
cv_params = {'learning_rate': [0.1,0.2,0.3],
             'max_depth': [1,3,5,7],
             'min_child_weight': [0.2,0.5,0.9,1,2,5],
             'n_estimators': [30,50]
             }

# 3. Define a dictionary of scoring metrics to capture
scoring = {'accuracy', 'precision', 'recall', 'f1', 'roc_auc'}

# 4. Instantiate the GridSearchCV object
xgb2 = GridSearchCV(xgb, cv_params, scoring=scoring, cv=4, refit='f1')

In [ ]:
%%time
# Fit the XGBoost model to the training data.
xgb2.fit(X_train, y_train) #Wall time: ~6min

In [ ]:
# Write pickle
write_pickle(path, xgb2, 'hr_xgb2')

In [ ]:
# Read in pickle
xgb2 = read_pickle(path, 'hr_xgb2')

In [ ]:
# Check best params
xgb2.best_params_

In [ ]:
# Check best F1 score on CV
xgb2.best_score_

In [ ]:
xgb2_cv_results = make_results('XGB CV 2', xgb2, 'f1')
tree_cv_results = pd.concat([tree_cv_results, xgb2_cv_results], axis=0)
tree_cv_results = tree_cv_results.reset_index(drop=True)
tree_cv_results

In [ ]:
# In case a row is to be dropped, input index number(s) in drop()
#tree_cv_results = tree_cv_results.drop([5], axis=0).reset_index(drop=True)
#tree_cv_results

Again, the scores dropped but the scores indicate that the model is a strong model. It appears that the second round, XGBoost model outperforms Decision Tree and Random Forest.

Use the best performing model to predict on the test set.

In [ ]:
# Get predictions on test data
xgb2_test_scores = get_scores('XGB CV Test 2', xgb2, X_test, y_test)
xgb2_test_scores

Based on the test scores, the second round XGBoost model appears to be a stable, well-performing model.

For this case study, the champion model to be chosen is the <strong>XGBoost</strong> model.

Additionally, plot a confusion matrix to visualize the performance of the model on test set.

In [ ]:
# Generate array of values for confusion matrix
preds = xgb2.best_estimator_.predict(X_test)
cm = confusion_matrix(y_test, preds, labels=xgb2.classes_)

# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                             display_labels=xgb2.classes_)
disp.plot(values_format='', cmap=plt.cm.YlGnBu)
plt.show()

The model predicts a little more false positives than false negatives. This means that at the process of predicting employees that will leave, there is a higher probability that the model might incorrectly identify employees as leaving when they are actually not, than the probability of the model incorrectly identifying employees as staying when they are actually leaving.

For further analysis, inspect the features with the most impact on an employee's decision to stay or leave.

## <span style="color:#808080">Feature Importance</span>


### <span style="color:#808080">Decision Tree</span>

For the decision tree model, examine the decision splits and the most important features.

In [ ]:
# Plot the decision tree split
plt.figure(figsize=(85,20))
plot_tree(tree2.best_estimator_, max_depth=6, fontsize=14, feature_names=X.columns, 
          class_names={0:'stayed', 1:'left'}, filled=True);
plt.show()

In [ ]:
# Sort features based on their importance using gini_importance
tree2_importances = pd.DataFrame(tree2.best_estimator_.feature_importances_, 
                                 columns=['gini_importance'], 
                                 index=X.columns
                                )
tree2_importances = tree2_importances.sort_values(by='gini_importance', ascending=False)

# Only extract the features with importances > 0
tree2_importances = tree2_importances[tree2_importances['gini_importance'] != 0]
tree2_importances

In [ ]:
# Create a barplot to visualize the decision tree feature importances
sns.barplot(data=tree2_importances, x="gini_importance", y=tree2_importances.index, orient='h', color='#3373a1')
plt.title("Decision Tree: Feature Importances for Employee Leaving", fontsize=12)
plt.ylabel("Feature")
plt.xlabel("Importance")
plt.show()

Based on the diagram, the decision tree model predicts that the most important features are `last_evaluation`, `number_project`, `tenure`, and `overworked`, in order, in determining the outcome variable, `left`.

### <span style="color:#808080">Random Forest</span>

Next, examine most important features for the Random Forest model.

In [ ]:
# Get feature importances
feat_impt = rf2.best_estimator_.feature_importances_

# Get indices of top 10 features
ind = np.argpartition(rf2.best_estimator_.feature_importances_, -10)[-10:]

# Get column labels of top 10 features 
feat = X.columns[ind]

# Filter `feat_impt` to consist of top 10 feature importances
feat_impt = feat_impt[ind]

y_df = pd.DataFrame({"Feature":feat,"Importance":feat_impt})
y_sort_df = y_df.sort_values("Importance")
fig = plt.figure()
ax1 = fig.add_subplot(111)

y_sort_df.plot(kind='barh',ax=ax1,x="Feature",y="Importance", color = "#3373a1")

ax1.set_title("Random Forest: Feature Importances for Employee Leaving", fontsize=12)
ax1.set_ylabel("Feature")
ax1.set_xlabel("Importance")

plt.show()

Based on the diagram, the random forest model also predicts that the most important features are `last_evaluation`, `number_project`, `tenure`, and `overworked`, in order, in determining the outcome variable, `left`.

### <span style="color:#808080">XGBoost</span>

Finally, examine most important features for the Random Forest model.

In [ ]:
# Get feature importances
feat_impt = xgb2.best_estimator_.feature_importances_

# Get indices of top 10 features
ind = np.argpartition(xgb2.best_estimator_.feature_importances_, -10)[-10:]

# Get column labels of top 10 features 
feat = X.columns[ind]

# Filter `feat_impt` to consist of top 10 feature importances
feat_impt = feat_impt[ind]

y_df = pd.DataFrame({"Feature":feat,"Importance":feat_impt})
y_sort_df = y_df.sort_values("Importance")
fig = plt.figure()
ax1 = fig.add_subplot(111)

y_sort_df.plot(kind='barh',ax=ax1,x="Feature",y="Importance", color='#3373a1')

ax1.set_title("XGBoost: Feature Importances for Employee Leaving", fontsize=12)
ax1.set_ylabel("Feature")
ax1.set_xlabel("Importance")

plt.show()

The champion model also predicts the same top four most important features but different order. Based on the XGBoost model, the most important features are `number_project`, `overworked`, `tenure`, and `last_evaluation`, in order, in determining the outcome variable, `left`.

# <span style="color:#35a455"><strong>EXECUTE STAGE</strong></span>

## <span style="color:#808080">Results</span>

The logistic regression model achieved 86% precision, 93% recall, 90% f1-score, and accuracy of 82%. On the other hand, the tree-based modeling achieved 86% precision, 93% recall, 90% f1-score, and accuracy of 82%, with XGBoost model being chosen as the champion model.

The factors with the greatest impact on an employee's decision to leave are (1) the number of projects that employee is assigned to, (2) whether that employee is overworked or not, (3) tenure in the company, and (4) the most recent evaluation score.


## <span style="color:#808080">Conclusions and Recommendations</span>

The results of this case study have shown that a huge portion of the company is overworked. To improve employee retention, this study recommends the following:

* Restrict the number of projects that the employees get assigned to. Set the limit to 5 projects.
* Re-evaluate project timelines and expectations, taking into account the amount of effort the employees can provide without overworking them. This can be done by assuming each employee can work for only 8 hours per day and has two weeks of vacation annually.
* Consider hiring more people so the employees need not work overtime.
* Continue and recalibrate the employee evaluation system to make sure those employees who contribute more or put in more effort have high evaluation scores.
* Investigate if employees are paid accordingly when they work overtime. If not, inform employees about company policies regarding overtime pay and make these policies explicit. If they are paid accordingly for working overtime, anticipate the effects on employee satisfaction when they are getting limited overtime, thus, lower overall pay as well. The decrease in satisfaction due to lower overall pay can be mitigated by implementing a salary increase for all employees, or those with satisfactory to high evaluation scores.
* Create an Award System that will reward employees upon reaching a certain number of years in the company.
* Conduct further investigation why four-year tenured employees have very low satisfaction levels. Also, consider giving them incentives when they reach the four-year mark in the company.
* Initiate company-wide and team-wide retrospectives and discussions to understand and address issues with company culture.

To further improve the results of this study, it is recommended to consider other factors that might affect an employee's decision to leave such as the distance between the company and the employee's residence, their salary growth in the past years, participation in activities that encourage work-life balance, and opportunities for growth in the company.